In [41]:
from sklearn.experimental import enable_iterative_imputer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, RepeatedStratifiedKFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import IterativeImputer

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [3]:
d_tr = pd.read_csv('/Users/antirrabia/Documents/01-GitHub/DataMining-_-/CSV/Titanic/train.csv', index_col='PassengerId')
d_te = pd.read_csv('/Users/antirrabia/Documents/01-GitHub/DataMining-_-/CSV/Titanic/test.csv', index_col=0)

In [4]:
d_tr.sample(7)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
271,0,1,"Cairns, Mr. Alexander",male,NaN,0,0,113798,31.0000,NaN,S
471,0,3,"Keefe, Mr. Arthur",male,NaN,0,0,323592,7.2500,NaN,S
472,0,3,"Cacic, Mr. Luka",male,38.0,0,0,315089,8.6625,NaN,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
841,0,3,"Alhomaki, Mr. Ilmari Rudolf",male,20.0,0,0,SOTON/O2 3101287,7.9250,NaN,S
345,0,2,"Fox, Mr. Stanley Hubert",male,36.0,0,0,229236,13.0000,NaN,S
394,1,1,"Newell, Miss. Marjorie",female,23.0,1,0,35273,113.2750,D36,C


In [8]:
d_te.sample(7)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
995,3,"Johansson Palmquist, Mr. Oskar Leander",male,26.0,0,0,347070,7.7750,NaN,S
1225,3,"Nakid, Mrs. Said (Waika Mary"" Mowad)""",female,19.0,1,1,2653,15.7417,NaN,C
1086,2,"Drew, Master. Marshall Brines",male,8.0,0,2,28220,32.5000,NaN,S
1285,2,"Gilbert, Mr. William",male,47.0,0,0,C.A. 30769,10.5000,NaN,S
1153,3,"Nilsson, Mr. August Ferdinand",male,21.0,0,0,350410,7.8542,NaN,S
1141,3,"Khalil, Mrs. Betros (Zahie Maria"" Elias)""",female,NaN,1,0,2660,14.4542,NaN,C
1110,1,"Widener, Mrs. George Dunton (Eleanor Elkins)",female,50.0,1,1,113503,211.5000,C80,C


In [16]:
classifier = Pipeline([('impute', IterativeImputer), 
                 ('model', 'passthrough')])

In [19]:
data = clean_data(d_tr)
data.sample(7)

,Sex_female,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Survived,Pclass,Age,Fare,Embarked,IsAlone
457,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,NaN,3,0.0,0
443,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1,2,3.0,1,0.0,1
585,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1,1,2.0,3,0.0,0
274,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1,3,NaN,0,2.0,1
24,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,3,1.0,2,0.0,0
885,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,3,4.0,2,2.0,0
625,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0,1,5.0,3,0.0,1


In [21]:
data.isna().sum()

Sex_female        0
Sex_male          0
Title_Master      0
Title_Miss        0
Title_Mr          0
Title_Mrs         0
Title_Other       0
Survived          0
Pclass            0
Age             177
Fare              0
Embarked          2
IsAlone           0
dtype: int64

In [23]:
X = data.drop(columns='Survived').copy()
y = data.Survived.copy()

In [38]:
y.columns

AttributeError: 'Series' object has no attribute 'columns'

In [29]:
model = LogisticRegression(max_iter=10000)
param_grid = [{ 
              'model__solver': ['newton-cg', 'lbfgs', 'liblinear'],
              'model__C': [0.01, 0.1, 1, 10, 100],
              'impute__imputation_order': ['ascending', 'descending', 'roman', 'arabic', 'random'],
              'impute__max_iter': [1,2,3,5,7,11,13,17,19]
              }]

In [30]:
classifier = Pipeline([('impute', IterativeImputer), 
                       ('model', model)])

In [42]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
gs = GridSearchCV(classifier, param_grid=param_grid, cv=5, n_jobs=-1)#, n_iter=20)

In [43]:
gs.fit(X, y)

TypeError: get_params() missing 1 required positional argument: 'self'

## Tools

In [18]:
def Fare(s):
    #s['FareBand'] = pd.qcut(s['Fare'], 4)
    s.loc[ s['Fare'] <= 7.91, 'Fare'] = 0
    s.loc[(s['Fare'] > 7.91) & (s['Fare'] <= 14.454), 'Fare'] = 1
    s.loc[(s['Fare'] > 14.454) & (s['Fare'] <= 31), 'Fare']   = 2
    s.loc[ s['Fare'] > 31, 'Fare'] = 3

    #Cambiar tipo de columna
    s['Fare'] = s['Fare'].astype(int)
    
    return s

def Age(s): #if it uses 2 times it will set all values to 1.0
            #recive a copy
            #does not touch nan values
    s.loc[(s.Age <= 13), 'Age'] = 1
    s.loc[(s.Age > 13) & (s.Age <= 18), 'Age'] = 2
    s.loc[(s.Age > 18) & (s.Age <= 30), 'Age'] = 3
    s.loc[(s.Age > 30) & (s.Age <= 50), 'Age'] = 4
    s.loc[(s.Age > 50), 'Age'] = 5
    
    return s

def FamilySize(s):
    s['FamilySize'] = s['SibSp'] +  s['Parch'] + 1
    
    s['FamilySize'] = s['FamilySize'].astype(int)
    #s['Survived'] = s['Survived'].astype(int)
    
    s['IsAlone'] = 0
    s.loc[s['FamilySize'] == 1, 'IsAlone'] = 1
    
    return s.drop(columns=['SibSp', 'Parch', 'FamilySize']).copy()

def Titles(s):
    #str.extract(' ([A-Za-z]+)\.')
    s['Title'] = s['Name'].str.extract(' ([A-Za-z]+)\.')
    s['Title'] = s['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr',                                       
                                    'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
    s['Title'] = s['Title'].replace(['Mlle', 'Ms'], 'Miss')
    s['Title'] = s['Title'].replace(['Mme'], 'Mrs')
    
    return s.drop(columns='Name').copy()

def SexEmbarkedTitle_to_OneHot(s):
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    #s.columns
    
    # ['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
    
    cat_features = ['Sex', 'Title']
    res_features = ['Survived', 'Pclass', 'Age', 'Fare', 'Embarked', 'IsAlone']
    
    transformer = ColumnTransformer([('cat', encoder, cat_features)], remainder='passthrough')
    
    transformer.fit(s)
    
    tranformed_names = transformer.named_transformers_['cat'].get_feature_names_out(cat_features)
    #tranformed_names
    
    new_names = np.append(tranformed_names, res_features)
    
    return pd.DataFrame(transformer.transform(s), columns=new_names)


# Para prepara el df para age y embarked prediction

def Age_Embarked(df):

    age_X_te = df.loc[df.Age.isna()].copy()
    embarked_X_te = df.loc[df.Embarked.isna()].copy()

    embarked_X_te.drop(columns=['Embarked'], inplace=True)
    age_X_te.drop(columns=['Age'], inplace=True)


    age_X_tr = df.dropna(subset=['Age', 'Embarked']).copy()
    age_y_tr = age_X_tr.Age


    embarked_X_tr = df.dropna(subset=['Age', 'Embarked']).copy()
    embarked_y_tr = embarked_X_tr.Embarked

    # to leave the df with all columns
    # age_X_tr.drop(columns=['Age'], inplace=True)
    # embarked_X_tr.drop(columns=['Embarked'], inplace=True)
    
    return (age_X_te, embarked_X_te, age_X_tr, age_y_tr, embarked_X_tr, embarked_y_tr)

def get_x_y(df, y):
    #resives an copy of a df and return X_tr, y_tr
    X_tr = df.drop(columns=[y])
    y_tr = df[y]
    
    return X_tr, y_tr



#temp01 = temp01.drop(columns=['Ticket', 'Cabin'])

def clean_data(df=None):
    '''This function uses Title(), Fare(), Age(), FamilySize()
       functions to clean up the Data Fram that it recives.
    '''
    
    d_clean = Titles(df)
    d_clean = Fare(d_clean)
    d_clean = Age(d_clean)
    d_clean = FamilySize(d_clean)

    #OneHot aplied
    d_clean.drop(columns=['Ticket', 'Cabin'], inplace=True)

    d_clean = SexEmbarkedTitle_to_OneHot(d_clean)
    
    d_clean['Embarked'] = d_clean['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2})#.astype(int)
        
    return d_clean


#Converting clean_data(df) function into one that 
#can be used in a pipeline
cleanDataFrame = FunctionTransformer(clean_data)
